In [1]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
test = pd.DataFrame({"pt": list("bbacab"), 
                     "peptide": ['abc', 'efg', 'abc', 'abc', 'xyx', 'xyz'],
                    "hla": [["A0","B1"],["A0","B1"],["C2","D2"],
                            ["A0", "D2", "A5"],["C2","D2"], ["A0","B1"]]})

test.set_index("pt", inplace=True)
test

,peptide,hla
pt,,
b,abc,"[A0, B1]"
b,efg,"[A0, B1]"
a,abc,"[C2, D2]"
c,abc,"[A0, D2, A5]"
a,xyx,"[C2, D2]"
b,xyz,"[A0, B1]"


In [3]:
test_hla = test[["hla"]].drop_duplicates()
hla_idx = test_hla.index

mlb = MultiLabelBinarizer(sparse_output=True)
test_hla = mlb.fit_transform(test_hla.hla)

hla_labs = ["hla-" + i for i in mlb.classes_]
# putting into sparse dataframe for easier index-based rejoining
test_hla = pd.DataFrame.sparse.from_spmatrix(test_hla, index=hla_idx, columns=hla_labs)

In [7]:
test_pep = test[["peptide"]].groupby("pt").agg({'peptide':lambda x: list(x)})
pep_idx = test_pep.index

mlb = MultiLabelBinarizer(sparse_output=True)
test_pep = mlb.fit_transform(test_pep.peptide)

pep_labs = ["mtpep-" + i for i in mlb.classes_]
test_pep = pd.DataFrame.sparse.from_spmatrix(test_pep, index=pep_idx, columns=pep_labs)

In [8]:
test_hla.join(test_pep)

,hla-A0,hla-A5,hla-B1,hla-C2,hla-D2,mtpep-abc,mtpep-efg,mtpep-xyx,mtpep-xyz
pt,,,,,,,,,
b,1,0,1,0,0,1,1,0,1
a,0,0,0,1,1,1,0,1,0
c,1,1,0,0,1,1,0,0,0
